In [1]:
import os
import sys 
from flask import Flask
from flask import request
from flask_api import status
import psycopg2
import json
import copy
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
datosBaseDeDatos = {"usuario":"postgres","password":"123456789","Host":"127.0.0.1","puerto":"5432","DB":"chatblocks"}

In [2]:
def abrirJson(name,path=os.getcwd()+"/" ):
    with open(path+name,encoding='utf-8') as data_file:  
        diccionario = json.load(data_file)
    return diccionario

In [3]:
def conexionBaseDeDatos(datosConexion):
    try:
        connection = psycopg2.connect(user = datosConexion["usuario"],
                              password = datosConexion["password"],
                              host = datosConexion["Host"],
                              port = datosConexion["puerto"],
                              database = datosConexion["DB"])    
        return connection
    except(Exception, psycopg2.Error) as error:
        print("Error al conectar con base de datos", error)

In [4]:
def consultaBots(connection,datos=False,consulta=False,tabla=False):
    #print("datos",datos)
    consultaBots = list()
    column_names = []
    data_rows = []
    diccionarioDatos = dict()
    consultas = list()
    cursor = connection.cursor()  
    cosulta =""    
    if datos:
        #print("intenatando consulta...")
        consulta = "select * from "+tabla+" where "        
        i=0
        for dato, valorDato in datos.items():
            if i==0:
                consulta = consulta+dato+"= '"+valorDato+"'"                
            else:             
                consulta = consulta+" and "+dato+"= '"+str(valorDato)+"'" 
            i=i+1
        #print(dato,":",valorDato)
        consulta= consulta + " ;" 
        #print(consulta)        
        #consulta = "select * from robots where name_robot = '"+datosBot["name_robot"]+"' and id_face = '"+ datosBot["id_face"]+"' and  type_blocki = '"+datosBot["type_blocki"]+"' and access_token= '"+datosBot["access_token"]+"' and id_user = '"+datosBot["id_user"]+"';"
    print("consulta:",consulta)
    cursor.execute(consulta)    
    column_names = [desc[0] for desc in cursor.description]  
    rows = cursor.fetchall()
    for row in rows:
        diccionarioDatos = dict()
        for i in range(len(column_names)):
            diccionarioDatos.update({column_names[i]:row[i]})
        consultaBots.append(diccionarioDatos)
    cursor.close()
    connection.close()  
    
    if len(consultaBots)==1:
        return consultaBots[0]
    else:
        return consultaBots

In [5]:
def insert(datos,tabla,connection):
    cursor = connection.cursor()
    values = ""
    datosInsert = ""
    i = 0;
    for dato, valorDato in datos.items():
        if i==0:
            values = values+dato 
            datosInsert = datosInsert+"'"+str(valorDato)+"' "                
        else:
            values = values+","+dato
            datosInsert =datosInsert+",'"+str(valorDato)+"' "
        i=i+1
        #print(dato,":",valorDato)
    #print("insert into "+tabla+" ( "+values+" ) values ("+datosInsert+" );" )
    insersion  = "insert into "+tabla+" ( "+values+" ) values ("+datosInsert+" );" 
    cursor.execute(insersion)
    connection.commit()
    cursor.close()
    connection.close()
    return datos

In [6]:
app = Flask(__name__)
@app.route('/bot', methods = ['POST'])
def postRobot():
    try:
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        datosBot = request.get_json()   
        print("datosBot",datosBot)
        tabla = "robots"
        insert(datosBot,tabla,connection)
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        con = consultaBots(connection,datos=datosBot,tabla=tabla)
        print("con",con)   
        return con
    except:
        return datosBot,status.HTTP_404_NOT_FOUND 
@app.route('/bloque', methods = ['POST'])
def postBloque():
    try:         
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        datosEnvio = request.get_json()   
        tabla = "bloque"+str(datosEnvio["blocktype"]).lower()
        insert(datosEnvio,tabla,connection)
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        con = consultaBots(connection,datos=datosEnvio,tabla=tabla)
        print("con",con)   
        return con
    except:
        return datosBot,status.HTTP_404_NOT_FOUND    
@app.route('/variable', methods = ['POST'])
def postVariable():
    try:         
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        datosVariable = request.get_json()   
        tabla = "variables"
        insert(datosVariable,tabla,connection)
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        con = consultaBots(connection,datos=datosVariable,tabla=tabla)
        print("con",con)   
        return con
    except:
        return datosVariable,status.HTTP_404_NOT_FOUND
@app.route('/access_token', methods = ['POST'])
def postAccessToken():
    try:         
        connection = conexionBaseDeDatos(datosBaseDeDatos)        
        datosVariable = request.get_json()   
        tabla = "robots"
        con = consultaBots(connection,datos=datosVariable,tabla=tabla)        
        listaAcessToken=list()
        if type(con)==list:
            for i in range(len(con)):
                print(con[i])
                listaAcessToken.append(con[i]["access_token"])
        else:
            listaAcessToken.append(con["access_token"])         
        print(listaAcessToken)
        return {"access_token":listaAcessToken}
    except:
        return datosVariable,status.HTTP_404_NOT_FOUND

In [7]:
if __name__ == "__main__":    
    app.run(host='localhost',port='8080')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Apr/2020 15:52:06] "POST /access_token HTTP/1.1" 200 -


consulta: select * from robots where access_token= 'EAAHNkscg2TcBAPwb5WSaxOM7IjDf57ZAoYu5TwDdUckaTK7dJc6nWq3fCaZBcKZBEZB3jHZCvbykRpYZAzpUHfIwmZCa45OcnXzND1FZBJNhaWyMmJMP6mKfJTr7bUJSVR8aZALoLyZC1M5mUUdIw54mVxVclTUecE2nSZCc3nTtheOKZAnLi8tdabkzHYFP1u8iwRoZD' ;
{'name_robot': 'tronquito', 'id_face': '2153123008120093', 'block_ini': 'Saludo2', 'type_blocki': 'informativo', 'access_token': 'EAAHNkscg2TcBAPwb5WSaxOM7IjDf57ZAoYu5TwDdUckaTK7dJc6nWq3fCaZBcKZBEZB3jHZCvbykRpYZAzpUHfIwmZCa45OcnXzND1FZBJNhaWyMmJMP6mKfJTr7bUJSVR8aZALoLyZC1M5mUUdIw54mVxVclTUecE2nSZCc3nTtheOKZAnLi8tdabkzHYFP1u8iwRoZD', 'api_nlp': 'Sin Api', 'id_robot': 121, 'id_user': 'oliverbp88@gmail.com'}
{'name_robot': 'tronquito', 'id_face': '2153123008120093', 'block_ini': 'Saludo2', 'type_blocki': 'informativo', 'access_token': 'EAAHNkscg2TcBAPwb5WSaxOM7IjDf57ZAoYu5TwDdUckaTK7dJc6nWq3fCaZBcKZBEZB3jHZCvbykRpYZAzpUHfIwmZCa45OcnXzND1FZBJNhaWyMmJMP6mKfJTr7bUJSVR8aZALoLyZC1M5mUUdIw54mVxVclTUecE2nSZCc3nTtheOKZAnLi8tdabkzHYFP1u8iwRoZD'